In [1]:
#импортируем необходимые модули
import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def recall(y_hat, y, thresh):
    return sum((((y - 1) * 2) + 1) == (y_hat > thresh)) / sum(y)
    
def precision(y_hat, y, thresh):
    return sum((((y - 1) * 2) + 1) == (y_hat > thresh)) / sum(y_hat > thresh)
    
def accuracy(y_hat, y, thresh):
    return sum(y == (y_hat > thresh)) / len(y)
    
def f1(y_hat, y, thresh):
    return 2 * recall(y_hat, y , thresh) * precision(y_hat, y, thresh) / (recall(y_hat, y , thresh) + precision(y_hat, y, thresh))


In [3]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

for i in range(10):
    print("Test {} passed".format(i))
    y_hat = np.random.random(30)
    y = np.random.random(30) > 0.7
    thresh = np.random.random()
    
    assert abs(recall(y_hat, y, thresh) - recall_score(y_true=y, y_pred=y_hat > thresh)) < 0.1
    assert abs(precision(y_hat, y, thresh) - precision_score(y_true=y, y_pred=y_hat > thresh)) < 0.1
    assert abs(accuracy(y_hat, y, thresh) - accuracy_score(y_true=y, y_pred=y_hat > thresh)) < 0.1
    assert abs(f1(y_hat, y, thresh) - f1_score(y_true=y, y_pred=y_hat > thresh)) < 0.1

Test 0 passed
Test 1 passed
Test 2 passed
Test 3 passed
Test 4 passed
Test 5 passed
Test 6 passed


/Users/amsavchenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


AssertionError: 

In [ ]:
y_hat = [1] * 100
y = [0] * 50
assert recall_score(y_hat, y) > 0.9

'''y_hat = #TODO предсказания, чтобы выбить precision 0.9 плохим классификатором
y = #TODO метки, чтобы выбить precision 0.9 плохим классификатором
t = #TODO порог, чтобы выбить precision 0.9 плохим классификатором

assert precision_score(y_hat, y, t) > 0.9
'''
y_hat = [0] * 100
y_hat.append(1)
y = [0] * 101

assert accuracy_score(y_hat, y) > 0.9

# Функция потерь для логистической регрессии (из материалов курса по нейронным сетям)

Давайте получим функцию потерь аналогичным образом, как мы это сделали с линейной регрессией.


логарифм правдоподобия
$$
\large 
\begin{array}{rcl} 
    \log P\left(\vec{y} \mid X, \vec{w}\right) 
        &=& \log \prod_{i=1}^{\ell} P\left(y = y_i \mid \vec{x_i}, \vec{w}\right) \\ 
        &=& \log \prod_{i=1}^{\ell} \sigma(y_i\vec{w}^T\vec{x_i}) \\ 
        &=& \sum_{i=1}^{\ell} \log \sigma(y_i\vec{w}^T\vec{x_i}) \\ 
        &=& \sum_{i=1}^{\ell} \log \frac{1}{1 + \exp^{-y_i\vec{w}^T\vec{x_i}}} \\ 
        &=& - \sum_{i=1}^{\ell} \log (1 + \exp^{-y_i\vec{w}^T\vec{x_i}}) 
\end{array}
$$
который и будем оптимизировать. 

Данная функция потерь 
$$
\large L(\vec{x},\vec{y},\vec{w}) = \sum_{i=1}^{\ell} \log (1 + \exp^{-y_i\vec{w}^T\vec{x_i}})
$$
называется логистичской функцией потерь или сокращенно logloss.

Дано:
- X - обучающее множество
- Y - метки классов для обучающего множества
- $\eta$ - скорость обучения

Алгоритм:
1. инициализируем $\vec{w}_0$ небольшими случайными значениями
2. выбираем случайный объект $x_i$ с меткой $y_i$
3. обновляем веса $ \vec{w}_{t+1} = \vec{w}_{t} - \eta \nabla L $
4. если условие остановки не выполнело, возвращаемся к шагу 2

Выпишем градиент функции потерь:
$$
    \nabla L = \left\{ - \frac{y_i \vec{x_i} }  { 1 + \exp^{y_i\vec{w}^T\vec{x_i}} } + \lambda \vec{w} \right\} 
$$

На практике, в качестве условия остановки, часто используют изменение функции потерь на тестовой выборке

In [ ]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

iris = datasets.load_iris()
selector = iris.target != 0 # ограничемся только двумя классами
X = iris.data[:, [1,3]][selector]  # и двумя переменными
y = iris.target[selector]
y[y==2] = 0 # конвертируем метки классов к 1/-1
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1, edgecolor='k')

In [ ]:
X_train = X[:80]
X_test = X[80:]

In [ ]:
X_train.shape

In [ ]:
polytrans = PolynomialFeatures(3)
X_train = polytrans.fit_transform(X_train)
X_test = polytrans.fit_transform(X_test)

In [ ]:
clf_1 = #TODO ОБучить логистическую регрессию на train с C=0.1
pred_1 = #TODO Предсказать логистической регрессией на трейне
pred_test = #TODO Предсказать логистической регрессией на тесте
#TODO Сранивть метрики

In [ ]:
#TODO обучим логистическую регрессию с разными C и посмотрим как меняются метрики на трейне и на тесте

# Пример текстовой классификации

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import grid_search
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

/Users/amsavchenko/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/amsavchenko/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amsavchenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amsavchenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None)
data.head()

,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [6]:
data.columns=['Review', 'Label']
data.head()

,Review,Label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
for i in range(0,len(data)):
    arr = data.iloc[i,0].split(' ')
    data.iloc[i,0] = ' '.join([str(porter_stemmer.stem(wordnet_lemmatizer.lemmatize(arr[i]))) for i in range(0, len(arr))])

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
import string
exclude = set(string.punctuation)

In [9]:
data['Review'] = data['Review'].apply(lambda x: ''.join(i for i in x if i not in exclude))

In [10]:
data.head()

,Review,Label
0,2 take around 10000 640x480 pictures,1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail and it worked ok...,1


In [14]:
vectorizer1 = CountVectorizer().fit(data['Review'])


In [16]:
features = vectorizer1.transform(data['Review'])

In [17]:
features

<2000x3750 sparse matrix of type '<class 'numpy.int64'>'
	with 29300 stored elements in Compressed Sparse Row format>

In [19]:
clf = LogisticRegression()
clf.fit(features, data.Label)

# коэфф 1/C 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

описание фильмов в векторы, сложим все векторы

выборка фильмов: k посмотрел и n не посмотрел (негативный сигнал) 
описание в вектор, для каждого человека логистичесскую или кнн предсказывает понравится фильм или нет
берем остальные и получаем предикт(вероятности), отрезаем топ-5 фильмов, которые ему рекомендуем

In [27]:
review = "awful experience"
vector_review = vectorizer1.transform([review])

In [21]:
vector_review

<1x3750 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [22]:
clf.predict(vector_review)

array([0])

In [26]:
vectorizer1.vocabulary_



{'take': 3263,
 'around': 322,
 '10000': 3,
 '640x480': 122,
 'picture': 2434,
 'downloaded': 1047,
 'trial': 3406,
 'version': 3536,
 'of': 2258,
 'computer': 739,
 'associate': 335,
 'ez': 1231,
 'firewall': 1297,
 'and': 273,
 'antivirus': 284,
 'fell': 1274,
 'in': 1662,
 'love': 1943,
 'with': 3677,
 'security': 2898,
 'system': 3253,
 'all': 243,
 'over': 2335,
 'again': 226,
 'the': 3308,
 'wrt54g': 3721,
 'plus': 2472,
 'hga7t': 1560,
 'is': 1749,
 'perfect': 2404,
 'solution': 3069,
 'if': 1630,
 'you': 3734,
 'need': 2163,
 'wireless': 3672,
 'coverage': 827,
 'wider': 3655,
 'area': 318,
 'or': 2305,
 'for': 1342,
 'hardwalled': 1517,
 'house': 1601,
 'wa': 3569,
 'my': 2137,
 'case': 587,
 'dont': 1035,
 'especially': 1166,
 'like': 1883,
 'how': 1603,
 'music': 2132,
 'file': 1280,
 'are': 317,
 'unstructured': 3484,
 'basically': 405,
 'they': 3319,
 'just': 1789,
 'dumped': 1073,
 'into': 1733,
 'one': 2279,
 'folder': 1332,
 'no': 2191,
 'organization': 2310,
 'might': 

In [12]:
data['Review'] = data['Review'].apply(lambda x: ' '.join(lemmatizer.lemmatize(i) for i in x.split()))

In [13]:
data.head()

,Review,Label
0,2 take around 10000 640x480 picture,1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music file are unst...,0
4,i wa using the cheapie pail and it worked ok u...,1


In [ ]:
vectors = [CountVectorizer(), TfidfVectorizer()]
parameters_grid = {
    'vectorizer__stop_words' : [nltk.corpus.stopwords.words('english'), 'english', None],
    'vectorizer__ngram_range' : [(1,2), (1,3)],
    'classifier__C':[0.25,0.5,1,2,5,10,15],
    'classifier__class_weight':['balanced', None],
    'classifier__max_iter':[70,100,200],
    'classifier__penalty':['l2','l1']
}
for vect in vectors:
        pipeline = Pipeline([('vectorizer', vect),('classifier',LogisticRegression())])
        grid_cv = grid_search.GridSearchCV(pipeline, parameters_grid, n_jobs=-1, scoring = 'accuracy', cv = 3)
        grid_cv.fit(data[0],data[1])
        print('Classifier: logReg, vectoizer: ',vect,', best_score: ',grid_cv.best_score_, grid_cv.best_params_)

In [ ]:
data_test = pd.read_csv('products_sentiment_test.tsv', sep='\t')
data_test.head()

In [ ]:
for i in range(0,len(data_test)):
    arr = data_test.iloc[i,1].split(' ')
    data_test.iloc[i,1] = ' '.join([str(porter_stemmer.stem(wordnet_lemmatizer.lemmatize(arr[i].lower()))) for i in range(0, len(arr))])
res = clf.predict(vectorizer.transform(data_test.text))

In [ ]:
#Appendix TODO together. Классификация отзывов о технике на русском